# **Задача 3. Параллельная сортировка с OpenMP**

Diana Kim ADA-2403M

**Практическое задание (25 баллов)**

Реализуйте алгоритм сортировки выбором с использованием OpenMP:
*   напишите последовательную реализацию алгоритма;
*   добавьте параллелизм с помощью директив OpenMP;
*   проверьте производительность для массивов размером 1 000 и 10 000 элементов.


In [11]:
%%writefile Task3.cpp

#include <iostream>     // для ввода и вывода
#include <omp.h>        // подключение OpenMP
#include <chrono>       // для измерения времени
#include <ctime>        // для time()
#include <cstdlib>      // для rand() и srand()

using namespace std;        // позволяет не писать std:: перед cout


// ПОСЛЕДОВАТЕЛЬНАЯ СОРТИРОВКА ВЫБОРОМ
void selectionSortSequential(int* arr, int n) {            // объявляет функцию последовательной сортировки
    for (int i = 0; i < n - 1; i++) {                 // проходит по всем элементам массива
        int minIndex = i;              // считает текущий элемент минимальным
        for (int j = i + 1; j < n; j++) {        // ищет минимальный элемент в оставшейся части
            if (arr[j] < arr[minIndex])             // проверяет, меньше ли текущий элемент
                minIndex = j;          // обновляет индекс минимального элемента
        }
        int temp = arr[i];                // сохраняет текущий элемент
        arr[i] = arr[minIndex];                 // меняет местами элементы
        arr[minIndex] = temp;             // завершает обмен
    }
}
// ПОСЛЕДОВАТЕЛЬНАЯ СОРТИРОВКА ВЫБОРОМ



// ПАРАЛЛЕЛЬНАЯ СОРТИРОВКА ВЫБОРОМ
void selectionSortParallel(int* arr, int n) {             // объявляет функцию параллельной сортировки
    for (int i = 0; i < n - 1; i++) {                  // проходит по всем элементам массива
        int minIndex = i;              // задает начальный минимальный индекс

#pragma omp parallel for                    // распараллеливает поиск минимума
        for (int j = i + 1; j < n; j++) {              // каждый поток проверяет свою часть массива

#pragma omp critical               // защищает доступ к minIndex
            {
                if (arr[j] < arr[minIndex])          // сравнивает элементы
                    minIndex = j;             // обновляет минимальный индекс
            }
        }

        int temp = arr[i];            // сохраняет текущий элемент
        arr[i] = arr[minIndex];                // меняет местами элементы
        arr[minIndex] = temp;            // завершает обмен
    }
}
// ПАРАЛЛЕЛЬНАЯ СОРТИРОВКА ВЫБОРОМ



int main() {
    const int sizes[2] = {1000, 10000};           // задает размеры массивов для тестирования

    for (int s = 0; s < 2; s++) {            // проходит по каждому размеру массива
        int N = sizes[s];            // сохраняет текущий размер массива
        int* arr1 = new int[N];               // создает массив для последовательной версии
        int* arr2 = new int[N];              // создает массив для параллельной версии
        srand((unsigned)time(0));          // инициализирует генератор случайных чисел

        for (int i = 0; i < N; i++) {             // заполняет массивы
            int value = rand() % 100000 + 1;                // генерирует случайное число
            arr1[i] = value;              // записывает число в первый массив
            arr2[i] = value;           // записывает число во второй массив
        }

        // ПОСЛЕДОВАТЕЛЬНАЯ ВЕРСИЯ
        auto start_seq = chrono::high_resolution_clock::now();                // запоминает начало времени
        selectionSortSequential(arr1, N);                  // выполняет последовательную сортировку
        auto end_seq = chrono::high_resolution_clock::now();                   // запоминает конец времени
        chrono::duration<double, milli> time_seq = end_seq - start_seq;           // вычисляет время
        // ПОСЛЕДОВАТЕЛЬНАЯ ВЕРСИЯ


        // ПАРАЛЛЕЛЬНАЯ ВЕРСИЯ
        auto start_par = chrono::high_resolution_clock::now();                 // запоминает начало времени
        selectionSortParallel(arr2, N);                    // выполняет параллельную сортировку
        auto end_par = chrono::high_resolution_clock::now();                     // запоминает конец времени
        chrono::duration<double, milli> time_par = end_par - start_par;              // вычисляет время
        // ПАРАЛЛЕЛЬНАЯ ВЕРСИЯ



        cout << "\nArray size: " << N << endl;            // выводит размер массива
        cout << "Sequential time: " << time_seq.count() << " ms" << endl;            // выводит время последовательной версии
        cout << "Parallel time: " << time_par.count() << " ms" << endl;           // выводит время параллельной версии
        cout << "OpenMP threads: " << omp_get_max_threads() << endl;         // выводит количество потоков OpenMP

        delete[] arr1;            // освобождает память первого массива
        delete[] arr2;             // освобождает память второго массива
    }

    return 0;
}

Writing Task3.cpp


In [12]:
!g++ Task3.cpp -fopenmp -O2 -o Task3
!./Task3


Array size: 1000
Sequential time: 0.65432 ms
Parallel time: 11.1307 ms
OpenMP threads: 2

Array size: 10000
Sequential time: 46.6822 ms
Parallel time: 1092.92 ms
OpenMP threads: 2
